In [1]:
import os
import sys
src_path = sys.path[0].replace("notebooks", "src")
# data_path = sys.path[0].replace("notebooks", "data")
if src_path not in sys.path:
    sys.path.append(src_path)

out_path = sys.path[0].replace("notebooks", "output")

import warnings
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import time

from profile_extraction import ProfileExtraction
from data_loader import UCSD_dataloader
from data_pool import DataPool

from battery_model import *
from optimizer import *

from exp_manager import ExperimentManager
import copy

g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# run extraction experiments

In [2]:
loader = UCSD_dataloader
loaded = loader(tstart=datetime(2015,1,1,0,0), tend=datetime(2019,12,31,0,0), delta=0.25, 
                bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=0.2, Pmax=10)
data = loaded.get_data()

In [3]:
class Extraction_ExperimentManager(ExperimentManager):
    def run_one_trial(self, params, save_fn):

        par = copy.deepcopy(self.default_params)
        par.update(params)

        tstart = par.get("tstart")
        tend = par.get("tend")
        pe = ProfileExtraction(data=data, tstart=tstart, tend=tend)

        pe_par = {k: par.get(k) for k in 
                  ["alg", "K", "dist_metric", "importance_func",
                   "incl_ev", "ev_how_to", "rand_seed"]}
        
        rp = pe.rep_profile_extraction(**pe_par)
        
        if self.save:
            rp.save(save_fn=save_fn, save_path=self.save_path)    

        stats = rp.stats
        return stats
    
    def init_default_params(self):
        self.default_params = {
            "tstart": datetime(2018, 1, 1, 0, 0),
            "tend": datetime(2019, 1, 1, 0, 0),
            "alg": "cluster",
            "K": 20,
            "dist_metric": None,
            "importance_func": None,
            "incl_ev": False,
            "ev_how_to": None,
            "incl_tou": None
        }

In [5]:
log_fn = os.path.join(out_path, "debug_test", "extraction_log_test.xlsx")
save_path = os.path.join(out_path, "debug_test")
em = Extraction_ExperimentManager(log_fn=log_fn, save_path=save_path, exp_prefix="Yi_TEST0309")
em.run(keys=["alg", "K", "dist_metric", "incl_ev", "ev_how_to"], num_trials=10)

g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Done, trial 10
Done, trial 11
Done, trial 12


g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Done, trial 13
Done, trial 14
Done, trial 15
Done, trial 16
Done, trial 17
Done, trial 18
DONE


# Use extracted profiles for battery sizing

In [3]:
from predictor import Predictor_tou_CAISO

In [4]:
params_sample = {
    "K": 96,
    "delta": 0.25,
    "S": 1, # number of scenarios (energy profiles). can omit, default 1
    "S_prob": None, 
    # if S != 1, params in "loads", "ev" should be list of length S
    ### loads
    "load_bld": 5 * np.sin(np.linspace(0, 2*np.pi, 96)) + 10,
    "load_pv": np.maximum(0, -np.linspace(-6,6,96)**2+20),
    "energy_price_buy": Predictor_tou_CAISO(None).get_prediction(datetime(2019, 1,1, 0, 0), 96, 0.25), # $/kWh 
    "energy_price_sell": None, # if None, sell is not allowed. can omit
    ### demand charge
    "dc_price": 0.3, # $/day. ref: 18 $/mon
    "dc_prev_max": None, # track p_grid_max in the same billing cycle. can omit
    ### battery (here I only consider one battery)
    "bat_capacity": None, # if none, capacity is optimized
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 100, # $/kWh, ref: Tesla Powerwall
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 5000, # cycles in lifetime
    # "reg_lambda": 0, # can omit, then reg_lambda = 0
    # "reg_term": "p_norm", # options: "p", "p_norm", "e", "e_norm", ...
    "deg_model": "throughput",  # valid values: "throughput", "Crate", "rainflow", "DOD"
    "deg_thres": None,
    "deg_lambda": None,
    "bat_soc_0": 0.5, # B0, BT have to be fractions (SoC indeed). 0.5 if omit
    "bat_soc_K": None, # if None, default is the same as bat_soc_0
    ### EVs
    "ev_I": 20,
    "ev_ta": np.linspace(0, 48, 20), # ta, td can be floats
    "ev_td": np.linspace(36, 96, 20),
    "ev_e_init": np.array([0]*20),
    "ev_e_targ": np.array([10]*20),
    "ev_capacity": None, # can omit, default as e_targ (useful only when aloow discharge)
    "ev_p_max": 6.6,
    "ev_p_min": 0, # can omit, default as 0
    "ev_efficacy": 0.98,
}

In [5]:
tstart, tend = datetime(2018, 1, 1, 0, 0), datetime(2019, 1, 1, 0, 0)
pe = ProfileExtraction(data=data, tstart=tstart, tend=tend)

In [6]:
# rp is a [RepProfiles] object, see src/profile/extraction.py
rp = pe.rep_profile_extraction(alg="cluster", K=20, incl_ev=True, ev_how_to="unif")
# convert profiles to params -> for battery optimizer
params = rp.profiles_to_params()

g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [7]:
params_sample["deg_model"] = "Crate"
opt_params = dict(params_sample)
opt_params.update(params)

In [15]:
bat_params = {
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 100, # $/kWh, ref: Tesla Powerwall
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 500, # cycles in lifetime
    # "reg_lambda": 0, # can omit, then reg_lambda = 0
    # "reg_term": "p_norm", # options: "p", "p_norm", "e", "e_norm", ...
    "deg_model": "rainflow",  # valid values: "throughput", "Crate", "rainflow", "DOD"
}

In [17]:
b_sample = Battery_base(**bat_params)
opt = Battery_optimizer(battery=b_sample)
# sol = opt.optimize_battery_size(params, strategy="optimal", reg_lambda=None, search_lim=(0,10))
sol = opt.optimize_battery_size(opt_params, strategy="optimal", mute=True, reg_lambda=0)

In [18]:
tco = sol["TCO"]
bat_capacity = sol["bat_capacity"]
tco, bat_capacity

(329.08054883244637, 1497.04052330263)

In [19]:
sol["eq_cycles"]

array([0.22461988, 0.15064948, 0.10017321, 0.14227421, 0.09799269,
       0.15446331, 0.1584085 , 0.09142298, 0.11905565, 0.17920531,
       0.10161344, 0.08046103, 0.13914102, 0.11380478, 0.17416872,
       0.09576623, 0.09141569, 0.15044978, 0.1929626 , 0.17289964])

# Run extraction & sizing (EnS)

In [5]:
loader = UCSD_dataloader
loaded = loader(tstart=datetime(2015,1,1,0,0), tend=datetime(2019,12,31,0,0), delta=0.25, 
                bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=0.2, Pmax=10)
data = loaded.get_data()

In [6]:
from predictor import Predictor_tou_CAISO

class EnS_ExperimentManager(ExperimentManager):
    def run_one_trial(self, params, save_fn):

        par = copy.deepcopy(self.default_params)
        par.update(params)

        tstart = par.get("tstart")
        tend = par.get("tend")
        pe = ProfileExtraction(data=data, tstart=tstart, tend=tend)

        pe_par = {k: par.get(k) for k in 
                  ["alg", "K", "dist_metric", "importance_func",
                   "incl_ev", "ev_how_to", "rand_seed"]}
        
        rp = pe.rep_profile_extraction(**pe_par)
        
        opt_params = dict(self.opt_params_sample)
        opt_params.update(rp.profiles_to_params())

        b_sample = Battery_base()
        opt = Battery_optimizer(battery=b_sample)
        # sol = opt.optimize_battery_size(params, strategy="optimal", reg_lambda=None, search_lim=(0,10))
        # FIXME: here are some hyperparameters, they need to be specified in self.default_params
        sol = opt.optimize_battery_size(opt_params, strategy="optimal", mute=True, reg_lambda=0)
        
        # TODO:
        # save the sol results ?

        stats = {
            "bat_capacity": sol["bat_capacity"],
            "TCO": sol["TCO"], # TODO: you can make more stats
        }

        return stats
    
    def init_default_params(self):
        self.default_params = {
            "tstart": datetime(2018, 1, 1, 0, 0),
            "tend": datetime(2019, 1, 1, 0, 0),
            "alg": "cluster",
            "K": 20,
            "dist_metric": None,
            "importance_func": None,
            "incl_ev": False,
            "ev_how_to": None,
            "incl_tou": None
        }

        self.opt_params_sample = {
            "K": 96,
            "delta": 0.25,
            "S": 1, # number of scenarios (energy profiles). can omit, default 1
            "S_prob": None, 
            # if S != 1, params in "loads", "ev" should be list of length S
            ### loads
            "load_bld": 5 * np.sin(np.linspace(0, 2*np.pi, 96)) + 10,
            "load_pv": np.maximum(0, -np.linspace(-6,6,96)**2+20),
            "energy_price_buy": Predictor_tou_CAISO(None).get_prediction(datetime(2019, 1,1, 0, 0), 96, 0.25), # $/kWh 
            "energy_price_sell": None, # if None, sell is not allowed. can omit
            ### demand charge
            "dc_price": 0.3, # $/day. ref: 18 $/mon
            "dc_prev_max": None, # track p_grid_max in the same billing cycle. can omit
            ### battery (here I only consider one battery)
            "bat_capacity": None, # if none, capacity is optimized
            "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
            "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
            "bat_price": 1000, # $/kWh, ref: Tesla Powerwall
            "bat_efficacy": 0.98, 
            "bat_life_0": 3650, # days.
            "bat_cycle_0": 10000, # cycles in lifetime
            "reg_lambda": 0, # can omit, then reg_lambda = 0
            "reg_term": "p_norm", # options: "p", "p_norm", "e", "e_norm", ...
            "bat_soc_0": 0.5, # B0, BT have to be fractions (SoC indeed). 0.5 if omit
            "bat_soc_K": None, # if None, default is the same as bat_soc_0
            ### EVs
            "ev_I": 20,
            "ev_ta": np.linspace(0, 48, 20), # ta, td can be floats
            "ev_td": np.linspace(36, 96, 20),
            "ev_e_init": np.array([0]*20),
            "ev_e_targ": np.array([10]*20),
            "ev_capacity": None, # can omit, default as e_targ (useful only when aloow discharge)
            "ev_p_max": 6.6,
            "ev_p_min": 0, # can omit, default as 0
            "ev_efficacy": 0.98,
        }

In [7]:
log_fn = os.path.join(out_path, "debug_test", "EnS_log_test.xlsx")
save_path = os.path.join(out_path, "debug_test")
em = EnS_ExperimentManager(log_fn=log_fn, save_path=save_path, save=False, exp_prefix="Yi_EnS_TEST0309")
em.run(keys=["alg", "K", "dist_metric", "incl_ev", "ev_how_to"], num_trials=10)

g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Done, trial 1
Done, trial 2
Done, trial 3


g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Done, trial 4
Done, trial 5
Done, trial 6
Done, trial 7
Done, trial 8
Done, trial 9
DONE
